Закрепление знаний

In [15]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
diab_data=pd.read_csv('data/diabetes_data.csv', sep=',')
diab_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 778 entries, 0 to 777
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               778 non-null    int64  
 1   Glucose                   778 non-null    int64  
 2   BloodPressure             778 non-null    int64  
 3   SkinThickness             778 non-null    int64  
 4   Insulin                   778 non-null    int64  
 5   BMI                       778 non-null    float64
 6   DiabetesPedigreeFunction  778 non-null    float64
 7   Age                       778 non-null    int64  
 8   Outcome                   778 non-null    int64  
 9   Gender                    778 non-null    object 
dtypes: float64(2), int64(7), object(1)
memory usage: 60.9+ KB


Задание 8.1
Начнём с поиска дубликатов в данных. Найдите все повторяющиеся строки в данных и удалите их. Для поиска используйте все признаки в данных. Сколько записей осталось в данных?

In [3]:
dupl_col=list(diab_data.columns)
mask1=diab_data.duplicated(subset=dupl_col)
diab_duplicates = diab_data[mask1]
print(f'Число найденных дубликатов: {diab_duplicates.shape[1]}')
diab_dedupped = diab_data.drop_duplicates(subset=diab_data)
print(f'Результирующее число записей: {diab_dedupped.shape[0]}')

Число найденных дубликатов: 10
Результирующее число записей: 768


Задание 8.2 Далее найдите все неинформативные признаки в данных и избавьтесь от них. В качестве порога информативности возьмите 0.95: удалите все признаки, для которых 95 % значений повторяются или 95 % записей уникальны. В ответ запишите имена признаков, которые вы нашли (без кавычек).

In [4]:
low_information_cols = [] 

# цикл по всем столбцам
for col in diab_dedupped.columns:
    # наибольшая относительная частота в признаке
    top_freq = diab_dedupped[col].value_counts(normalize=True).max()
    # доля уникальных значений от размера признака
    nunique_ratio = diab_dedupped[col].nunique() / diab_data[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')

# дропнем весь этот мусор через drop
information_diab_data=diab_dedupped.drop(low_information_cols, axis=1)
print(f'Результирующее число признаков: {information_diab_data.shape[1]}')
print(f'Результирующее число записей: {information_diab_data.shape[0]}')
information_diab_data.head()

Gender: 100.0% одинаковых значений
Результирующее число признаков: 9
Результирующее число записей: 768


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,98,58,33,190,34.0,0.430,43,0
1,2,112,75,32,0,35.7,0.148,21,0
2,2,108,64,0,0,30.8,0.158,21,0
3,8,107,80,0,0,24.6,0.856,34,0
4,7,136,90,0,0,29.9,0.210,50,0


Задание 8.3
Попробуйте найти пропуски в данных с помощью метода isnull().

Спойлер: ничего не найдёте. А они есть! Просто они скрыты от наших глаз. В таблице пропуски в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI обозначены нулём, поэтому традиционные методы поиска пропусков ничего вам не покажут. Давайте это исправим!

Замените все записи, равные 0, в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI на символ пропуска. Его вы можете взять из библиотеки numpy: np.nan.

Какая доля пропусков содержится в столбце Insulin? Ответ округлите до сотых.

In [5]:
def nan_function(x):
    return np.nan if x == 0 else x
information_diab_data["Glucose"]=information_diab_data["Glucose"].apply(nan_function)
information_diab_data["BloodPressure"]=information_diab_data["BloodPressure"].apply(nan_function)
information_diab_data["SkinThickness"]=information_diab_data["SkinThickness"].apply(nan_function)
information_diab_data["Insulin"]=information_diab_data["Insulin"].apply(nan_function)
information_diab_data["BMI"]=information_diab_data["BMI"].apply(nan_function)
information_diab_data.isnull().mean().round(2).sort_values(ascending=False)

Insulin                     0.49
SkinThickness               0.30
BloodPressure               0.05
Glucose                     0.01
BMI                         0.01
Pregnancies                 0.00
DiabetesPedigreeFunction    0.00
Age                         0.00
Outcome                     0.00
dtype: float64

Задание 8.4. Удалите из данных признаки, где число пропусков составляет более 30 %. Сколько признаков осталось в ваших данных (с учетом удаленных неинформативных признаков в задании 8.2)?

In [6]:
#задаем минимальный порог: вычисляем 70% от числа строк
thresh=information_diab_data.shape[0]*0.7
#удаляем столбцы, в которых более 30% (100-70) пропусков
information_diab_data=information_diab_data.dropna(thresh=thresh, axis=1)
#отображаем результирующую долю пропусков
information_diab_data.isnull().mean()
print(information_diab_data.shape[1])

8


Задание 8.5. Удалите из данных только те строки, в которых содержится более двух пропусков одновременно. Чему равно результирующее число записей в таблице?

In [7]:
n=information_diab_data.shape[1]
information_diab_data=information_diab_data.dropna(thresh=n-2, axis=0)
print(f'Результирующее число записей: {information_diab_data.shape[0]}')

Результирующее число записей: 761


Задание 8.6. В оставшихся записях замените пропуски на медиану. Чему равно среднее значение в столбце SkinThickness? Ответ округлите до десятых.

In [14]:
values={
    'Pregnancies':information_diab_data['Pregnancies'].median(),
    'Glucose':information_diab_data['Glucose'].median(),
    'BloodPressure':information_diab_data['BloodPressure'].median(),
    'SkinThickness':information_diab_data['SkinThickness'].median(),
    'BMI':information_diab_data['BMI'].median(),
    'DiabetesPedigreeFunction':information_diab_data['DiabetesPedigreeFunction'].median(),
    'Age':information_diab_data['Age'].median(),
    'Outcome':information_diab_data['Outcome'].median(),
}
information_diab_data=information_diab_data.fillna(values)
information_diab_data.mean()
display(round(information_diab_data['SkinThickness'].mean(),1))

29.1

Задание 8.7 Сколько выбросов найдёт классический метод межквартильного размаха в признаке SkinThickness?

In [19]:
def outliers_iqr(data, feature):
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned
outliers, cleaned = outliers_iqr(information_diab_data, 'SkinThickness')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу Тьюки: 87
Результирующее число записей: 674


Задание 8.8 Сколько выбросов найдёт классический метод z-отклонения в признаке SkinThickness?

In [25]:
def outliers_z_score(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - 3 * sigma
    upper_bound = mu + 3 * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_z_score(information_diab_data, 'SkinThickness', log_scale=False)
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу z-отклонения: 4
Результирующее число записей: 757


Задание 8.9 Найдите сначала число выбросов в признаке DiabetesPedigreeFunction с помощью классического метода межквартильного размаха. Затем найдите число выбросов в этом же признаке в логарифмическом масштабе (при логарифмировании единицу прибавлять не нужно!). Какова разница между двумя этими числами (вычтите из первого второе)?

In [26]:
def outliers_iqr(data, feature):
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned
outliers, cleaned = outliers_iqr(information_diab_data, 'DiabetesPedigreeFunction')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу Тьюки: 29
Результирующее число записей: 732


In [27]:
def outliers_iqr_mod(data, feature, left=3, right=3, log_scale=True):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x<lower_bound) | (x> upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_iqr_mod(information_diab_data, 'DiabetesPedigreeFunction')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу Тьюки: 0
Результирующее число записей: 761
